In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
from sklearn.metrics import classification_report , accuracy_score
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB

## 1. Data Loading and Exploration

In [ ]:
# Loading the dataset
df = pd.read_csv("/content/drive/MyDrive/CompanyReviews.csv")

In [ ]:
df.head()

,Unnamed: 0,review_description,rating,company
0,0,رائع,1,talbat
1,1,برنامج رائع جدا يساعد على تلبيه الاحتياجات بشك...,1,talbat
2,2,التطبيق لا يغتح دائما بيعطيني لا يوجد اتصال با...,-1,talbat
3,3,لماذا لا يمكننا طلب من ماكدونالدز؟,-1,talbat
4,4,البرنامج بيظهر كل المطاعم و مغلقه مع انها بتكو...,-1,talbat


In [ ]:
# Droping the last column
df = df.drop(columns=['Unnamed: 0','company'])
df.head()

,review_description,rating
0,رائع,1
1,برنامج رائع جدا يساعد على تلبيه الاحتياجات بشك...,1
2,التطبيق لا يغتح دائما بيعطيني لا يوجد اتصال با...,-1
3,لماذا لا يمكننا طلب من ماكدونالدز؟,-1
4,البرنامج بيظهر كل المطاعم و مغلقه مع انها بتكو...,-1


In [ ]:
# Droping any row with rating = 0
df = df.drop(df[df['rating'] == 0].index)
df

,review_description,rating
0,رائع,1
1,برنامج رائع جدا يساعد على تلبيه الاحتياجات بشك...,1
2,التطبيق لا يغتح دائما بيعطيني لا يوجد اتصال با...,-1
3,لماذا لا يمكننا طلب من ماكدونالدز؟,-1
4,البرنامج بيظهر كل المطاعم و مغلقه مع انها بتكو...,-1
...,...,...
40028,الابليكيشن يدخلنا فى متاهات لا نفهم منها شيء ....,-1
40029,تجربة سيئة جدا منصحش اى حد بتحميل التطبيق النظ...,-1
40035,بعد ما اخدت الباقة الاولى ب ١٦٠ جنيه ٣٠ رحلة ب...,-1
40037,الخدمة جيدة ولكن يرجى تعديل سلوك بعض السائقين ...,1


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 38121 entries, 0 to 40042
Data columns (total 2 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   review_description  38120 non-null  object
 1   rating              38121 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 893.5+ KB


In [ ]:
# Check if there is null value
df.isnull().sum()

review_description    1
rating                0
dtype: int64

In [ ]:
# Drop the null values
df.dropna(subset=['review_description'], inplace=True)

In [ ]:
df.isnull().sum()

review_description    0
rating                0
dtype: int64

In [ ]:
df.shape

(38120, 2)

In [ ]:
# Number of negative and positive reviews
df.rating.value_counts()

 1    23921
-1    14199
Name: rating, dtype: int64

## 2. Data Preprocessing

In [ ]:
# Necessary libraries for preprocessing
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
nltk.download('punkt')
stop_words = set(stopwords.words('arabic'))
print(stop_words)

{'شمال', 'ذواتا', 'اتخذ', 'منها', 'بكما', 'شين', 'لكنَّ', 'رُبَّ', 'أول', 'شباط', 'اثنان', 'سبعمائة', 'أنتم', 'بمن', 'انقلب', 'خامس', 'صهٍ', 'عَدَسْ', 'أولاء', 'إي', 'أنت', 'حبذا', 'واهاً', 'كليهما', 'صراحة', 'مكانَك', 'هلم', 'تسعمائة', 'ست', 'ثمانية', 'أنا', 'إحدى', 'فإذا', 'أبٌ', 'أيلول', 'قاطبة', 'بَلْهَ', 'ثمَّ', 'فيه', 'كأيّن', 'كلما', 'ستون', 'آناء', 'أفٍّ', 'ذلك', 'أنشأ', 'علم', 'ماي', 'ثمّة', 'أمس', 'أى', 'س', 'إلا', 'زاي', 'حمو', 'أضحى', 'أنتما', 'أينما', 'لدن', 'بي', 'هناك', 'ولا', 'هَاتانِ', 'ا', 'نيف', 'ريث', 'مرّة', 'وَيْ', 'خاصة', 'حيث', 'إلّا', 'ث', 'حين', 'أف', 'تسعين', 'ألا', 'نوفمبر', 'والذين', 'أيّ', 'معاذ', 'عليه', 'مائة', 'هيهات', 'بل', 'لكن', 'فيفري', 'أرى', 'شتان', 'هاته', 'ثمنمئة', 'اثنين', 'إياكم', 'ليت', 'تَيْنِ', 'ميم', 'دواليك', 'تِه', 'ثمّ', 'خمس', 'ارتدّ', 'لن', 'كأنما', 'ذانك', 'ديسمبر', 'طَق', 'أصلا', 'أوشك', 'أيا', 'أربع', 'إياهم', 'عوض', 'أُفٍّ', 'كذلك', 'تسع', 'علًّ', 'حدَث', 'ممن', 'إليكنّ', 'ظ', 'هَجْ', 'هَاتِي', 'حبيب', 'ثلاثاء', 'لا سيما', 'آب', '

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
# pip install demoji

In [ ]:
# Remove emojis from the 'review_description' column
import demoji
df['review_description'] = df['review_description'].apply(lambda text: demoji.replace(text, ''))

In [ ]:
# Remove punctuations
import string
df['review_description'] = df['review_description'].apply(lambda text: ''.join(char for char in text if char not in string.punctuation))

In [ ]:
# Tokinzation
from nltk.tokenize import word_tokenize
df['review_description'] = df['review_description'].apply(lambda text: word_tokenize(text))

In [ ]:
df.head()

,review_description,rating
0,[رائع],1
1,"[برنامج, رائع, جدا, يساعد, على, تلبيه, الاحتيا...",1
2,"[التطبيق, لا, يغتح, دائما, بيعطيني, لا, يوجد, ...",-1
3,"[لماذا, لا, يمكننا, طلب, من, ماكدونالدز؟]",-1
4,"[البرنامج, بيظهر, كل, المطاعم, و, مغلقه, مع, ا...",-1


In [ ]:
# Remove stop words
stop_words = set(stopwords.words('arabic'))
df['review_description'] = df['review_description'].apply(lambda words: [word for word in words if word not in stop_words])

In [ ]:
# Normalization
from nltk.stem import ISRIStemmer
stemmer = ISRIStemmer()
df['review_description'] = df['review_description'].apply(lambda words: [stemmer.stem(word) for word in words])

In [ ]:
df['review_description'] = df['review_description'].apply(lambda words: ' '.join(words))
df.head()

,review_description,rating
0,رئع,1
1,رنمج رئع جدا سعد لبه حيج شكل سرع,1
2,طبق غتح دائ يعط وجد تصل شبكةمع انه الن عند تمم...,-1
3,لمذ يمك طلب ماكدونالدز؟,-1
4,رنمج ظهر طعم غلق انه بتك فتح بقل كده كتر شهر,-1


## 3. Feature Engineering

In [ ]:
# Handcraft feature extraction using TF-IDF
tfidf_vectorizer = TfidfVectorizer()
X = tfidf_vectorizer.fit_transform(df['review_description'])
y = df['rating']

## 4. Model Building

In [ ]:
# Splitting the train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Building the naive bayes model
nb_model = MultinomialNB()
nb_model.fit(X_train, y_train)

MultinomialNB()



## 5. Model Evaluation

In [ ]:
# Predict on the test
y_pred = nb_model.predict(X_test)

# Evaluate the classifier
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred, target_names=['Positives', 'Negatives',])
print(f"Accuracy: {accuracy:.2f}")
print(report)

Accuracy: 0.86
              precision    recall  f1-score   support

   Positives       0.84      0.78      0.81      2879
   Negatives       0.87      0.91      0.89      4745

    accuracy                           0.86      7624
   macro avg       0.86      0.84      0.85      7624
weighted avg       0.86      0.86      0.86      7624



In [ ]:
# Classify new review
new_text = " وصلني الطعام متأخرا والاكل بارد"

# Remove emojis from the new review
new_text = demoji.replace(new_text, '')

# Remove punctuation from the new review
new_text = ''.join(char for char in new_text if char not in string.punctuation)

# Tokenize the new review
new_text = word_tokenize(new_text)

# Remove stop words from the new review
new_text = [word for word in new_text if word not in stop_words]

# Normalize the new review (stemming)
new_text = [stemmer.stem(word) for word in new_text]

new_text = ' '.join(new_text)


In [ ]:
# Transform the new text using TF-IDF
new_text = tfidf_vectorizer.transform([new_text])

In [ ]:
# Predict the rating for the new text
predicted_rating = nb_model.predict(new_text)
if predicted_rating == 1:
  print("Predicted Label: Positive")
else:
  print("Predicted Label: Negative")

Predicted Label: Negative


## 6. Conclusion
- This code demonstrates a sentiment analysis using an arabic corpus about applications reviews that are either negative or positive.

- Including data preprocessing, feature engineering using handcrafted (TF-IDF), model building, and evaluation.

- The choice of a Multinomial Naive Bayes model for sentiment analysis is common and often performs well for tasks like sentiment analysis or rating prediction

- The gained accuarcy is 86%

## 7. References
https://www.kaggle.com/datasets/fahdseddik/arabic-company-reviews/data
